In [ ]:
#| default_exp paramviewer

# Exploring your remote data in a breeze 

> Instantly find and access all your Nextcloud data files

In order to to make use of `fairdatanow`, you will need a Nextcloud server url, a user name and a password to access a folder on a Nextcloud server. The recommended way to use these credentials in Jupyter notebooks is to store username and password as environment variables on your system. You can retrieve them with the `os.getenv()` function. In this way you avoid typing them directly in the notebook, which is not save if you need to share these notebooks with others.  

To get started you need to import the `DataViewer` class from the package and instantiate it with the Nextcloud `configuration` dictionary. Depending on the amount of files in the cloud storage it might take some time to build the interactive file table. In this example we want to create a table for more than 6000 files from the Falnama project which takes about 12 seconds. 

In [ ]:
from fairdatanow import DataViewer
import os

In [ ]:
configuration = {
    'url': "https://laboppad.nl/falnama-project", 
    'user':    os.getenv('NC_AUTH_USER'),
    'password': os.getenv('NC_AUTH_PASS')
}

In [ ]:
dv = DataViewer(configuration)
dv

DataViewer(bytes_amount=np.int64(12164564536), columns=['path', 'size', 'modified'], data=                                                   path       size    ext  \
0                                      falnama-project/  205.8 GiB          
1                   falnama-project/.attachments.12065/  749.2 KiB          
2     falnama-project/.attachments.12065/Falnama_por...  749.2 KiB   .jpg   
3                             falnama-project/.jupyter/    1.7 KiB          
4        falnama-project/.jupyter/desktop-settings.json   32 Bytes  .json   
...                                                 ...        ...    ...   
6384                falnama-project/OPENDATA/xrays.html    2.7 MiB  .html   
6385                          falnama-project/README.md  642 Bytes    .md   
6386                      falnama-project/Uploadfolder/   22 Bytes          
6387     falnama-project/Uploadfolder/firstdownload.txt   22 Bytes   .txt   
6388  falnama-project/filetree-laboppad-falnama_2025...  496.2 KiB   .txt   

         byte_size                  modified  isdir  
0     220952091744 2025-08-08 13:01:35+00:00   True  
1           767156 2024-10-11 08:28:52+00:00   True  
2           767156 2024-10-11 08:28:52+00:00  False  
3             1734 2024-10-18 12:22:13+00:00   True  
4               32 2024-10-18 12:22:13+00:00  False  
...            ...                       ...    ...  
6384       2796717 2025-08-01 11:19:27+00:00  False  
6385           642 2025-02-05 10:02:18+00:00  False  
6386            22 2025-07-23 09:55:21+00:00   True  
6387            22 2025-07-22 09:10:09+00:00  False  
6388        508080 2025-05-08 08:20:33+00:00  False  

[6389 rows x 6 columns], extensions=[], filtered_data=                                                   path       size  \
262   falnama-project/05. Photos Objects/01. Photos ...  122.9 MiB   
263   falnama-project/05. Photos Objects/01. Photos ...  893.0 KiB   
1431  falnama-project/07. Results/01. Xray-images (R...   10.1 MiB   
1432  falnama-project/07. Results/01. Xray-images (R...    9.8 MiB   
1433  falnama-project/07. Results/01. Xray-images (R...    9.8 MiB   
...                                                 ...        ...   
6008  falnama-project/OPENDATA/RMA_xrays/10_jpgs/718...    9.5 MiB   
6009  falnama-project/OPENDATA/RMA_xrays/10_jpgs/718...    9.9 MiB   
6010  falnama-project/OPENDATA/RMA_xrays/10_jpgs/718...   10.2 MiB   
6011  falnama-project/OPENDATA/RMA_xrays/File_inform...  102 Bytes   
6384                falnama-project/OPENDATA/xrays.html    2.7 MiB   

                      modified  
262  2024-10-10 20:45:20+00:00  
263  2024-10-10 20:57:48+00:00  
1431 2024-10-20 14:14:01+00:00  
1432 2024-10-20 14:14:08+00:00  
1433 2024-10-20 14:14:24+00:00  
...                        ...  
6008 2024-10-20 14:13:23+00:00  
6009 2024-10-20 14:12:14+00:00  
6010 2024-10-20 14:12:21+00:00  
6011 2025-08-05 13:47:45+00:00  
6384 2025-08-01 11:19:27+00:00  

[74 rows x 3 columns], name='DataViewer00121', search='xray', show_directories=False, show_filters=False)

If we scroll through this first selection of 74 file paths that contain the search string `xray`, we see that the interactive table contains different kinds of files related to the x-ray images. By selecting the `Show Filters` additional possibilities for further narrowing down the list can be selected to shown only the specific files we currently need. You can try this yourself. 

:::{.callout-note} 

Note that in the documentation interactivity is limited. 

:::

We can now select rows by Shift and Ctrl clicking from this interactive table. Rows that are selected will be colored blue. These selected files can then be downloaded with the `.download_selected()` method onto your local machine into a local cache folder. Downloading is skipped if the selected files are already present locally. The local file paths in our cache folder are returned in the `files` list for further processing.   

In [ ]:
files = dv.download_selected() 

Ready with downloading 2 selected remote files to local cache: /home/frank/.cache/fairdatanow                                                                      


## FUNCTIONS 

In [ ]:
#| export 

import panel as pn
import param
import pandas as pd
import nc_py_api
import re
import os
from panel.viewable import Viewer
from humanize import naturalsize
from pathlib import Path
import time

pn.extension("tabulator")

In [ ]:
#| export 

class DataViewer(Viewer):
    'Explore (y)our Nextcloud project data files.'
    
    # DataFrames
    data = param.DataFrame()
    filtered_data = param.DataFrame()
    # list filters
    columns = param.ListSelector(default=["path", "size", "modified"])
    extensions = param.ListSelector(default=[])
    # typed filters
    search = param.String(default="xray")
    bytes_amount = param.Integer()
    show_directories = param.Boolean(default=False)
    show_filters = param.Boolean(default=False)

    #non param attributes
    nc_py_api.options.NPA_NC_CERT = False 

    def __init__(self, configuration, subdir=None, **params):
        'Initalize a DataViewer instance.'
        
        super().__init__(**params)
        self.nc = self._create_connector(configuration)
        # load all param attributes where necessary
        self.data = self._load_dataframe(subdir)
        self.param.columns.objects = self.data.columns.to_list()
        self.param.extensions.objects = self.data['ext'].unique()

    def _create_connector(self, configuration):
        # parse configuration 
        m = re.match('(^https://[^/]+/)(.*)', configuration['url'])
        nextcloud_url, self.cache_dir = m.groups()
        nc_auth_user = configuration['user']
        nc_auth_pass = configuration['password']
        
        return nc_py_api.Nextcloud(nextcloud_url=nextcloud_url,
                                   nc_auth_user=nc_auth_user,
                                   nc_auth_pass=nc_auth_pass)

    @pn.cache()
    def _load_dataframe(self, subdir):
        if subdir is None:
            subdir = self.cache_dir

        fsnodes = self.nc.files.listdir(subdir, depth=-1, exclude_self=False)

        data = [{'path': fsnode.user_path,
                'size': naturalsize(fsnode.info.size, True),
                'ext': os.path.splitext(fsnode.user_path)[1].lower(),
                'byte_size': fsnode.info.size,
                'modified': fsnode.info.last_modified,
                'isdir': fsnode.is_dir
                } for fsnode in fsnodes]
        
        return pd.DataFrame(data)
       
    def download_selected(self, cache_dir=None):
        '''Download selected files (blue rows) from `table` to default local cache directory. 
        
        A custom `cache_dir` can be specified. '''
        
        # create cache path 
        if cache_dir is None: 
            cache_path = Path.home().joinpath('.cache', 'fairdatanow')
        else: 
            cache_path = Path.home().joinpath('.cache', cache_dir)
    
        os.makedirs(cache_path, exist_ok=True)

        remote_data = self.data.iloc[self._file_table.selected_dataframe.index.tolist()]

        # obtain remote paths and remote timestamps 
        local_path_list = []
       
        for i, [remote_path, remote_modified, remote_isdir] in enumerate(zip(remote_data['path'].tolist(), remote_data['modified'].tolist(), remote_data['isdir'].tolist())): 
    
            # only download actual files 
            if not remote_isdir:   
                remote_directory = os.path.dirname(remote_path)
                local_directory = cache_path.joinpath(remote_directory) # I guess this will not yet work for Windows
                
                # create directory structure inside cache 
                os.makedirs(local_directory, exist_ok=True) 
            
                # get remote epoch time  
                remote_modified_epoch_time = remote_modified.timestamp()
            
                # construct corresponding local path 
                local_path = cache_path.joinpath(remote_path) 
                local_path_list.append(str(local_path))
                
            
                # check if local file exists and if modification times are similar 
                is_local = local_path.exists()  
            
                is_similar = False 
                local_modified_epoch_time = None 
                if is_local: 
                    local_modified_epoch_time = os.stat(local_path).st_mtime
                    if local_modified_epoch_time == remote_modified_epoch_time: 
                        is_similar = True 
                        
                # download from nextcloud 
                if not is_similar: 
                    print(f'[{i+1}/{len(remote_data)}] Timestamps do no match: {remote_modified_epoch_time} vs {local_modified_epoch_time}', end='\r')
                    print(f'[{i+1}/{len(remote_data)}] Downloading to: {local_path}                                                       ' , end='\r')
                      
                    # write to cache 
                    with open(local_path, 'bw') as fh: 
                        self.nc.files.download2stream(remote_path, fh) 
                        
                    # adjust last modified timestamp 
                    now = int(time.time())
                    os.utime(local_path, (now, remote_modified_epoch_time)) 
                    
        print(f"Ready with downloading {len(remote_data)} selected remote files to local cache: {cache_path}                                                                      ")

        return local_path_list
    
    @param.depends("data", "columns", "search", "extensions", "show_directories", watch=True)
    def _update_filtered_data(self):
        # set the base df for readability and a non-watched variable
        df = self.data
        # search the dataframe in column path if it contains the search string
        df = df[df["path"].str.contains(self.search)]
        # filter to only include the extensions that are selected
        if self.extensions:
            df = df[df["ext"].isin(self.extensions)]
        # save the total bytes_size in the bytes_amount variable
        self.bytes_amount = df[df["isdir"] == False]["byte_size"].sum()
        # if show_directories is turned off exclude them from the df
        if not self.show_directories:
            df = df[df["isdir"] == False]
        # only select the columns from the column selector
        self.filtered_data = df[self.columns]
    
    @param.depends("filtered_data")
    def number_of_rows(self):
        return f"Showing {len(self.filtered_data)} out of {len(self.data)} rows | Total size: {naturalsize(self.bytes_amount, True)}"

    @param.depends("show_filters")
    def make_widgetbox(self):
        if self.show_filters:
            self.filter_menu = pn.WidgetBox('# Filters',
                                            pn.widgets.MultiChoice.from_param(self.param.columns),
                                            pn.widgets.MultiChoice.from_param(self.param.extensions),
                                            pn.widgets.Checkbox.from_param(self.param.show_directories)
                                           )
            return self.filter_menu
        self.filter_menu = None
        
    
    def __panel__(self):
        self._file_table = pn.widgets.Tabulator(self.param.filtered_data, height=350, pagination=None, show_index=False, selectable=True, disabled=True)
        return pn.Column(
            pn.Row(pn.widgets.TextInput.from_param(self.param.search), 
                   pn.widgets.Checkbox.from_param(self.param.show_filters),
                   self.make_widgetbox
                  ),
            self._file_table,
            self.number_of_rows
        )